# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yashvaishnav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yashvaishnav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    """
    inputs:
    messages
       
    Returns:
    list of words into numbers of same meaning
    """
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
def eval_metrics(ArrayL, ArrayP, col_names):
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    

In [20]:
# Calculate evaluation metrics for training set
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)

print(eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.990267   0.991863  0.995448  0.993652
request                 0.987091   0.996161  0.928444  0.961111
offer                   0.998566   1.000000  0.705263  0.827160
aid_related             0.986374   0.994873  0.972497  0.983558
medical_help            0.988576   0.996302  0.860703  0.923552
medical_products        0.992470   1.000000  0.851815  0.919978
search_and_rescue       0.994416   0.993590  0.814361  0.895091
security                0.995953   0.989051  0.780980  0.872786
military                0.995594   0.994614  0.869702  0.927973
water                   0.994416   1.000000  0.913217  0.954640
food                    0.995390   0.998111  0.960891  0.979147
shelter                 0.992777   0.998748  0.919885  0.957696
clothing                0.997951   0.992647  0.876623  0.931034
money                   0.996568   1.000000  0.855292  0.922002
missing_people          0.997234   1.000

In [21]:
# Calculate evaluation metrics for test set
Y_test_pred = pipe.predict(X_test)

eval_metrics0 = eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.802367   0.843826  0.909402  0.875388
request                 0.883510   0.790997  0.439286  0.564868
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.741048   0.722272  0.602837  0.657172
medical_help            0.919932   0.484848  0.061657  0.109402
medical_products        0.953435   0.673077  0.109034  0.187668
search_and_rescue       0.976794   0.562500  0.058824  0.106509
security                0.980483   0.000000  0.000000  0.000000
military                0.967420   0.703704  0.085202  0.152000
water                   0.944675   0.780000  0.187500  0.302326
food                    0.929461   0.813239  0.475138  0.599826
shelter                 0.931612   0.813084  0.300518  0.438840
clothing                0.985400   0.600000  0.061856  0.112150
money                   0.978638   0.600000  0.042553  0.079470
missing_people          0.987245   0.000

In [22]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.764792
request                   0.171892
offer                     0.004534
aid_related               0.417243
medical_help              0.080068
medical_products          0.050446
search_and_rescue         0.027816
security                  0.018096
military                  0.033041
water                     0.064239
food                      0.112302
shelter                   0.088904
clothing                  0.015560
money                     0.023206
missing_people            0.011449
refugees                  0.033618
death                     0.045874
other_aid                 0.132396
infrastructure_related    0.065506
transport                 0.046143
buildings                 0.051214
electricity               0.020440
tools                     0.006109
hospitals                 0.010873
shops                     0.004610
aid_centers               0.011872
other_infrastructure      0.044222
weather_related           0.280352
floods              

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
# Define performance metric for use in grid search scoring object
def perform_metric(Y_T, Y_P):
    """Median F1 score for all classifiers
    
    inputs:
    Y_T: array. Array containing ArrayL labels.
    Y_P: array. Array containing ArrayP labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

### This takes a while to run

In [24]:
# grid search

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipe, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(74)
T_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.127, total=  35.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.110, total=  35.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.115, total=  36.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  5.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.153, total=  29.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.160, total=  29.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.160, total=  30.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.116, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vec

[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.165, total=  55.8s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.132, total=  55.6s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.141, total=  55.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5, score=0.216, total=  47.5s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, 

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 50.8min finished


In [25]:
#grid search results
T_model.cv_results_

{'mean_fit_time': array([27.61548702, 23.58777372, 27.96730312, 22.93114368, 55.39306124,
        45.26894291, 55.79999304, 45.4797283 , 24.79149199, 21.66640814,
        23.4649663 , 20.98384778, 44.58539677, 39.08501808, 44.1311926 ,
        37.4480656 , 22.54718995, 20.6393942 , 21.54416498, 19.66050458,
        40.25510192, 36.82695341, 39.07299391, 34.44030333]),
 'std_fit_time': array([0.26013814, 0.12902099, 0.37241319, 0.29752232, 0.22826761,
        0.72105078, 0.73842418, 1.77857054, 1.05544227, 0.10989888,
        0.10364863, 0.12537212, 0.23782144, 0.21330336, 0.18688452,
        0.19222951, 0.07139557, 0.06383181, 0.12935219, 0.06739355,
        0.26639054, 0.20870047, 0.46749838, 0.16862085]),
 'mean_score_time': array([ 7.68746837,  7.04871837,  7.63337501,  7.09826231, 11.96501939,
        10.60245617, 11.73090164, 10.59458907,  8.12793628,  6.99770284,
         7.61334443,  7.01815502, 11.76693193, 10.09308147, 11.66546822,
        10.2842923 ,  7.97118203,  6.99848962

In [26]:
# top mean score
np.max(T_model.cv_results_['mean_test_score'])

0.21796015611425212

In [27]:
# Parameters for top mean score
T_model.best_params_


{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 10,
 'tfidf__use_idf': False,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
# evaluating metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(Y_test), tuned_pred_test, col_names)
print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.809590   0.838909  0.928931  0.881628
request                 0.891809   0.796296  0.499107  0.613611
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.757338   0.710084  0.693916  0.701907
medical_help            0.920547   0.506579  0.148362  0.229508
medical_products        0.954972   0.659091  0.180685  0.283619
search_and_rescue       0.977870   0.695652  0.104575  0.181818
security                0.980175   0.000000  0.000000  0.000000
military                0.968034   0.741935  0.103139  0.181102
water                   0.959428   0.811475  0.475962  0.600000
food                    0.945597   0.794586  0.689227  0.738166
shelter                 0.939911   0.767045  0.466321  0.580021
clothing                0.985861   0.608696  0.144330  0.233333
money                   0.979099   0.608696  0.099291  0.170732
missing_people          0.987398   0.500

In [29]:
# summary of 1st model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.941434,0.545535,0.176633,0.228930
std,0.058766,0.311056,0.238942,0.263613
min,0.741048,0.000000,0.000000,0.000000
25%,0.930536,0.391343,0.016146,0.031281
50%,0.954511,0.600000,0.058824,0.106509
75%,0.980406,0.789115,0.310987,0.439462
max,0.996465,1.000000,0.909402,0.875388


In [30]:
# summary of tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.945430,0.556759,0.239997,0.295066
std,0.055305,0.314854,0.275218,0.286459
min,0.757338,0.000000,0.000000,0.000000
25%,0.941294,0.460000,0.014572,0.028571
50%,0.959428,0.695652,0.112033,0.193548
75%,0.980559,0.780816,0.470003,0.590011
max,0.996465,1.000000,0.928931,0.881628


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
# using SVM inste ad 
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv2 = GridSearchCV(pipe2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(77)
T_model2 = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.4min
[CV] clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


[CV]  clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.4min
[CV] clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 19.2min remaining:    0.0s


[CV]  clf__estimator__C=1, clf__estimator__degree=3, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.4min
[CV] clf__estimator__C=10, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 21.5min remaining:    0.0s


[CV]  clf__estimator__C=10, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.8min
[CV] clf__estimator__C=10, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__C=10, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.8min
[CV] clf__estimator__C=10, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__C=10, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.8min
[CV] clf__estimator__C=10, clf__estimator__degree=2, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__C=10, clf__estimator__degree=2, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.000, total= 2.4min
[CV] clf__estimator__C=10, clf__estimator__degree=2, cl

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 67.7min finished


In [32]:
# grid search results
T_model2.cv_results_

{'mean_fit_time': array([100.79459023,  97.05863945,  95.92652734, 114.96100084,
         96.82942669,  95.72683231, 135.52718973,  97.22322369,
         95.14110033]),
 'std_fit_time': array([0.84848527, 0.52813645, 0.49461939, 0.28684776, 0.41033756,
        0.20308257, 1.29025647, 0.18485533, 0.42341499]),
 'mean_score_time': array([45.68554211, 45.4884096 , 45.33674184, 50.54041243, 45.42993752,
        45.20480498, 57.34199667, 45.46698268, 45.15871962]),
 'std_score_time': array([0.14818545, 0.10372381, 0.07531002, 0.04815436, 0.09377946,
        0.10819078, 0.13214101, 0.13191711, 0.11317324]),
 'param_clf__estimator__C': masked_array(data=[1, 1, 1, 10, 10, 10, 100, 100, 100],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__degree': masked_array(data=[1, 2, 3, 1, 2, 3, 1, 2, 3],
              mask=[False, False, False, False, False, False, False, Fa

median always zero therefore lets evaluate metrics

In [33]:
# Calculate  evaluation metrics for test set
tuned_pred_test2 = T_model2.predict(X_test)
#todo calculate the eval metrics 
eval_metrics2 = eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

                        Accuracy  Precision  Recall        F1
related                 0.763332   0.763332     1.0  0.865784
request                 0.827878   0.000000     0.0  0.000000
offer                   0.996465   0.000000     0.0  0.000000
aid_related             0.588290   0.000000     0.0  0.000000
medical_help            0.920240   0.000000     0.0  0.000000
medical_products        0.950669   0.000000     0.0  0.000000
search_and_rescue       0.976487   0.000000     0.0  0.000000
security                0.980944   0.000000     0.0  0.000000
military                0.965729   0.000000     0.0  0.000000
water                   0.936069   0.000000     0.0  0.000000
food                    0.888735   0.000000     0.0  0.000000
shelter                 0.911019   0.000000     0.0  0.000000
clothing                0.985093   0.000000     0.0  0.000000
money                   0.978331   0.000000     0.0  0.000000
missing_people          0.987398   0.000000     0.0  0.000000
refugees

SVM doesnt work ,we will be using RANDOM FOREST *FTW*

### 9. Export your model as a pickle file

In [34]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.